In [1]:
import os
import subprocess
import numpy
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# 通过后端异步处理
# 作为热身，考虑一个简单问题：生成一个随机矩阵并将其相乘，让我们在Numpy和Pytorch张量中都这样做，看看它们的区别，Pytorch的tensor是在GPU上定义的

In [9]:
# GPU计算热身
device = torch.device('cpu')
a = torch.randn(size=(1000, 1000), device=device)
b = torch.mm(a, a)

with d2l.Benchmark('numpy'):
    for _ in range(10):
        a = numpy.random.normal(size=(1000, 1000))
        b = numpy.dot(a, a)

with d2l.Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)

numpy: 0.2640 sec
torch: 0.1698 sec


In [4]:
# 通过pytorch的基准输出比较快乐几个数量级，numpy点积是在cpu上执行的，而pytorch矩阵乘法是在gpu上执行的，后者的速度要快的多，但巨大的时间差距表明一定还有其他原因。默认情况下，gpu操作在pytorch中是异步的，强制pytorch在返回之前返程所有计算，这种强制说明了之前发生的情况：计算是由后端执行，而前端将控制权返回给了python

In [6]:
with d2l.Benchmark():
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)
    torch.cuda.synchronize(device)

Done: 0.0047 sec


In [7]:
x = torch.ones((1, 2), device=device)
y = torch.ones((1, 2), device=device)
z = x*y+2
z

tensor([[3., 3.]], device='cuda:0')

In [8]:
x = torch.ones((1, 2))
y = torch.ones((1, 2))
z = x*y+2
z

tensor([[3., 3.]])